In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager


In [5]:

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("https://www.lamudi.pk/search/lahore/homes-for-sale-in-dha-phase-6-1448/?location_ids=1448,1450,1451")

In [7]:
property_location = []
property_prices = []
property_size = []
bed_count = []
bath_count = []
property_description = []


In [26]:

options = Options()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Base URL
base_url = "https://www.lamudi.pk/search/lahore/homes-for-sale-in-dha-phase-6-1448/?location_ids=1448,1450,1451"


property_list = []


for page in range(1, 6):
    url = f"{base_url}&page={page}"
    driver.get(url)
    
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "u-mb32")))
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    listings = soup.find_all("div", class_="card-horizontal_cardHorizontal__OzX3s")

    for listing in listings:
        
        try:
            location = listing.find("div", class_="card-horizontal_cardSubTitle__3FJyc").text.strip()
        except AttributeError:
            location = None
        
        try:
            price = listing.find("h4", class_="card-horizontal_cardTitle__dHyrR price_price__oxYSY heading_h4__5Kb_J").text.strip()
        except AttributeError:
            price = None
        
        specs = listing.find("div", class_="card-horizontal_cardSpecs__3U-Ym")
        if specs:
            specs_divs = specs.find_all("div") 
            try:
                area = specs_divs[0].text.strip() if len(specs_divs) > 0 else None
            except AttributeError:
                area = None

            try:
                beds = specs_divs[1].text.strip() if len(specs_divs) > 1 else None
            except AttributeError:
                beds = None

            try:
                baths = specs_divs[2].text.strip() if len(specs_divs) > 2 else None
            except AttributeError:
                baths = None
        else:
            area, beds, baths = None, None, None

        property_list.append({
            "Location": location,
            "Price": price,
            "Area": area,
            "Beds": beds,
            "Baths": baths
        })

driver.quit()

df = pd.DataFrame(property_list)

df.to_csv("lamudi_listings.csv", index=False)

print(df.tail(10))

                               Location            Price      Area Beds Baths
115            DHA Phase 6, DHA Defence   PKR 7.99 Crore   1 Kanal    5     6
116            DHA Phase 6, DHA Defence     PKR 57 Crore   2 Kanal    7     7
117            DHA Phase 6, DHA Defence     PKR 13 Crore   1 Kanal    5     6
118            DHA Phase 7, DHA Defence    PKR 9.5 Crore   1 Kanal    5     7
119            DHA Phase 7, DHA Defence   PKR 7.15 Crore   1 Kanal    5     6
120  DHA Phase 6 - Block L, DHA Phase 6   PKR 24.5 Crore   1 Kanal    6     7
121            DHA Phase 6, DHA Defence  PKR 13.99 Crore   1 Kanal    5     6
122  DHA Phase 6 - Block G, DHA Phase 6   PKR 11.5 Crore   1 Kanal    5     6
123  DHA Phase 6 - Block L, DHA Phase 6    PKR 7.9 Crore   1 Kanal    5     6
124            DHA Phase 7, DHA Defence      PKR 5 Crore  10 Marla    4     5
